In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#define image size and batch size
IMG_SIZE=224
BATCH_SIZE=32

In [ ]:
#define data generators for,train, validation and the test sets
train_datagen=ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)


train_generator=train_datagen.flow_from_directory('/content/drive/MyDrive/Brain_Tumor_Detection/Brain_Tumor_Detection/train',
    target_size=(IMG_SIZE,IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)
val_generator=train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Brain_Tumor_Detection/Brain_Tumor_Detection/train',
    target_size=(IMG_SIZE,IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)
test_datagen=ImageDataGenerator(rescale=1./255)
test_generator=test_datagen.flow_from_directory('/content/drive/MyDrive/Brain_Tumor_Detection/Brain_Tumor_Detection/test',
    target_size=(IMG_SIZE,IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)

Found 2400 images belonging to 2 classes.
Found 600 images belonging to 2 classes.
Found 60 images belonging to 1 classes.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#define the model
model=keras.Sequential([
    layers.Conv2D(32,(3,3),activation='relu',input_shape=(IMG_SIZE,IMG_SIZE,3)),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(128,(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(128,activation='relu'),
    layers.Dense(1,activation='sigmoid')
    ])

In [ ]:
#Compile the model
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
history = model.fit(train_generator,validation_data=val_generator,epochs=5)

Epoch 1/5
75/75 [==============================] - 719s 10s/step - loss: 0.5761 - accuracy: 0.7829 - val_loss: 0.3862 - val_accuracy: 0.8233
Epoch 2/5
75/75 [==============================] - 319s 4s/step - loss: 0.2472 - accuracy: 0.8971 - val_loss: 0.1480 - val_accuracy: 0.9517
Epoch 3/5
75/75 [==============================] - 292s 4s/step - loss: 0.1176 - accuracy: 0.9663 - val_loss: 0.0751 - val_accuracy: 0.9633
Epoch 4/5
75/75 [==============================] - 292s 4s/step - loss: 0.0694 - accuracy: 0.9796 - val_loss: 0.0705 - val_accuracy: 0.9733
Epoch 5/5
75/75 [==============================] - 314s 4s/step - loss: 0.0351 - accuracy: 0.9908 - val_loss: 0.0242 - val_accuracy: 0.9967


In [ ]:
model.save("Model.h5","labe.txt")

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

#load the saved model
model=load_model('/content/Model.h5')

#load and preprocess the test image
test_image_path='/content/drive/MyDrive/Brain_Tumor_Detection/Brain_Tumor_Detection/test/pred/pred10.jpg'
img=image.load_img(test_image_path,target_size=(224,224))
img_array=image.img_to_array(img)
img_array=np.expand_dims(img_array ,axis=0)  #batch dimension
img_array /=255.  #normalise the pixel values
#make predictions
prediction=model.predict(img_array)
#print the prediction
if prediction < 0.5:
  print("prediction: No tumor (Probability:",prediction[0][0],")")
else:
  print("prediction: tumor present (Probability:",prediction[0][0],")")

1/1 [==============================] - 0s 141ms/step
prediction: tumor present (Probability: 0.99971914 )
